In [7]:
import time
import sqlite3
import requests
import datetime

import pandas as pd
import shioaji as sj
import re
from shioaji import TickFOPv1, Exchange
from exchange_calendars import get_calendar

import mplfinance as mpf

from FuncBase import *

In [8]:
import os 
import shioaji as sj
from dotenv import load_dotenv

#永豐金API登入
load_dotenv()
api = sj.Shioaji(simulation=True)
api.login(os.getenv('API_KEY'),os.getenv('SECRET_KEY'),  
          contracts_cb= lambda security_type: print(f"{repr(security_type)} fetch done"))

#連結DB Browser
connection = sqlite3.connect('work_data.db')

Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done
<SecurityType.Stock: 'STK'> fetch done
<SecurityType.Future: 'FUT'> fetch done
<SecurityType.Option: 'OPT'> fetch done


In [50]:
import time
def get_ticks(connection, api, dates, codes = str, is_Futures = False):

    tw_calendar = get_calendar('XTAI')

    #name tickers and correct time stamp

    if is_Futures:
        
        #若為期貨只取英文代號作為code name
        code = ''.join(char for char in codes if char.isalpha())

        if dates.hour > 15:
            date = dates+datetime.timedelta(days=1)
        
        else:
            date = dates

    else:
        code = codes
        date = dates
        

    try:    
        sql = "SELECT * FROM ticks WHERE code = '{}' and ts BETWEEN '{}' AND '{}' ".format(code,
                                                                                 date,
                                                                                 date+datetime.timedelta(days=1))
        df = pd.read_sql(sql, connection, parse_dates = ['ts'], index_col=['ts'])
    except:
        df = pd.DataFrame()
    
    if not df.empty:
        return df, True
    
    if is_Futures: #若為期貨
            ticks = api.ticks(
                contract = api.Contracts.Futures.get(code)[codes],  # For futures, use the Futures contract
                date=date.strftime('%Y-%m-%d') 
                )

    else: #若為證券
            ticks = api.ticks(
            contract=api.Contracts.Stocks[code],  # For stocks, use the Stocks contract
                date=date.strftime('%Y-%m-%d')
            )    
    df = pd.DataFrame({**ticks})

    df.ts = pd.to_datetime(df.ts)
    df['code'] = code
    df = df.set_index('ts')

    return df, False

In [51]:
dates = pd.to_datetime('2024-11-7')
df, in_db = get_ticks(connection, api, dates, '2330')


print(in_db)
df

False


,ask_price,ask_volume,close,tick_type,bid_price,bid_volume,volume,code
ts,,,,,,,,
2024-11-07 09:00:02.297525,1055.0,992,1050.0,2,1050.0,2168,3176,2330
2024-11-07 09:00:02.498126,1055.0,1045,1055.0,1,1050.0,2417,1,2330
2024-11-07 09:00:02.760760,1055.0,1045,1055.0,1,1050.0,2470,1,2330
2024-11-07 09:00:04.424996,1055.0,1181,1055.0,1,1050.0,2471,1,2330
2024-11-07 09:00:04.451920,1055.0,1180,1055.0,1,1050.0,2471,1,2330
...,...,...,...,...,...,...,...,...
2024-11-07 13:24:53.890919,1070.0,743,1065.0,2,1065.0,1451,2,2330
2024-11-07 13:24:53.928181,1070.0,743,1065.0,2,1065.0,1449,2,2330
2024-11-07 13:24:54.023237,1070.0,744,1065.0,2,1065.0,1448,1,2330


In [9]:
def update_ticks(connection, api, daily_target, is_Futures = False):
    main_df = pd.DataFrame()

    tw_calendar = get_calendar('XTAI')

    for date, codes in daily_target.items():

        day_trading_codes = [code for code in codes ]

        print('正在更新{}逐筆成交資料，總共{}檔標的，為{}'.format(date.strftime('%Y/%m/%d'), len(day_trading_codes),day_trading_codes))

        for code in day_trading_codes:
            df, in_db = get_ticks(connection, api, date, code, is_Futures)

            if df is not None and in_db:
                main_df = pd.concat([main_df, df], sort = False)
                time.sleep(1)

            prev_trading_date = tw_calendar.previous_close(date).date()
            prev_df, prev_in_db = get_ticks(connection, api, date, code, is_Futures)

            if prev_df is not None and not prev_in_db:
                main_df = pd.concat([main_df, prev_df], sort = False)
                time.sleep(1)
    
    if not main_df.empty:
        try:
            main_df = main_df.reset_index().drop_duplicates().set_index('ts')
            main_df.to_sql('ticks', connection, if_exists='append')
            print("Data stored successfully.")
        except Exception as e:
            print("Failed to store data:", e)
    return main_df



In [12]:
daily_targets ={
    pd.to_datetime('2024-11-24'): ['2317', '2330'],
    pd.to_datetime('2024-11-25'): ['2317', '2330'],
    pd.to_datetime('2024-11-26'): ['2317', '2330']
                }

In [13]:
connection = sqlite3.connect('work_data.db')
update_ticks(connection, api, daily_targets)

正在更新2024/11/24逐筆成交資料，總共2檔標的，為['2317', '2330']
正在更新2024/11/25逐筆成交資料，總共2檔標的，為['2317', '2330']
正在更新2024/11/26逐筆成交資料，總共2檔標的，為['2317', '2330']
Data stored successfully.


,bid_price,ask_volume,close,ask_price,volume,tick_type,bid_volume,code
ts,,,,,,,,
2024-11-25 09:00:01.845766,205.5,610.0,206.0,206.0,1010.0,1.0,133.0,2317
2024-11-25 09:00:01.907715,205.5,610.0,205.5,206.0,1.0,2.0,131.0,2317
2024-11-25 09:00:02.006432,205.5,611.0,205.5,206.0,1.0,2.0,133.0,2317
2024-11-25 09:00:02.017008,205.5,611.0,205.5,206.0,1.0,2.0,152.0,2317
2024-11-25 09:00:02.209495,205.5,503.0,206.0,206.0,11.0,1.0,152.0,2317
...,...,...,...,...,...,...,...,...
2024-11-26 13:24:51.893964,1015.0,3546.0,1015.0,1020.0,2.0,2.0,444.0,2330
2024-11-26 13:24:52.413046,1015.0,3545.0,1020.0,1020.0,1.0,1.0,445.0,2330
2024-11-26 13:24:53.787166,1015.0,3545.0,1015.0,1020.0,4.0,2.0,443.0,2330


In [8]:
def retry_request(url, payloads, headers):
    
    for i in range(3):
    
        try:
            return requests.get(url, params= payloads, headers=headers)
        
        except:
            print('發生錯誤，請等待一分種後再嘗試')
            time.sleep(60)
    
    return None

def get_daily_prices(date, connection):

    try:
        df = pd.read_sql("select * from daily_prices where 日期 = '{}'".format(date),
                         connection,
                         parse_dates= ['日期'],
                         index_col= ['證券代號','日期'])
    except:
        df = pd.DataFrame()
    
    if not df.empty:
        return df, True
    #INDB TRUE
    #先在資料庫裡抓取每日收盤價，沒有資料則利用爬蟲重新抓取
    
    url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX'

    payloads = {
        'response': 'html',
        'date': date.strftime('%Y%m%d'),
        'type': 'ALLBUT0999'
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.80 Safari/537.36'
    }

    response = retry_request(url, payloads, headers)

    try:
        df = pd.read_html(response.text)[-1]
    
    except:
        print('{} 找不到資料'.format(
            date.strftime('%Y%m%d')))
        
        return None, False
    #INDB False
    df.columns = df.columns.get_level_values(2)
    df['漲跌價差'] = df['漲跌價差'].where(df['漲跌(+/-)'] != '-', -df['漲跌價差'])
    df.drop(['證券名稱','漲跌(+/-)'],inplace=True, axis=1)
    df['日期'] =  pd.to_datetime(date).strftime('%Y-%m-%d 00:00:00')
    df = df.set_index(['證券代號','日期'])
    df = df.apply(pd.to_numeric, errors = 'coerce')
    df.drop(df[df['收盤價'].isnull()].index, inplace=True)
    df['昨日收盤價'] = df['收盤價'] - df['漲跌價差']
    df['股價震幅'] = (df['最高價'] - df['最低價']) / df['昨日收盤價']*100

    return df, False



In [32]:
def update_daily_prices(start_date, end_date, connection):

    tw_calendars = get_calendar("XTAI")

    main_df = pd.DataFrame()

    for date in pd.date_range(start_date, end_date):

        if date not in tw_calendars.opens:
            continue
    
        print(date)

        try:
            sql = "SELECT * FROM daily_price WHERE 日期 ='{}' ".format(date)
            df = pd.read_sql(sql, connection, index_col=['證券代號'])
        except:
            df = pd.DataFrame()

        if not df.empty:
            print('{}:Data exist.'.format(date.strftime('%Y%m%d')))
        
        else:
        
            df, in_db = get_daily_prices(date, connection)

            if df is not None:
                main_df = pd.concat([main_df,df],axis = 0)
                print('{} 每日收盤行情抓取完成，等待15秒'.format(date.strftime('%Y%m%d')))
                time.sleep(15)


    if not main_df.empty:
        main_df.to_sql('daily_price', connection, if_exists='append')
        
        return main_df

    

in_db判斷式失效

In [37]:
def update_historical_data(start_date, end_date, codes, connection, api):

    tw_calendar = get_calendar('XTAI')

    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)

    print('正在更新每日收盤行情...')

    update_daily_prices(tw_calendar.previous_close(start).date(), end, connection)

    daily_target= {}

    for date in pd.date_range(start, end):

        if date not in tw_calendar.opens:
            continue
        
        elif (
            date == datetime.datetime.now().date() and
            datetime.datetime.now().hour < 15
        ):
            break

        daily_target[date] = [codes] if isinstance(codes, str) else codes
    print(daily_target)
    
    
    print('正在更新逐筆交易...')
        
    update_ticks(connection, api, daily_target)

    print('股市資料更新完成')


In [47]:
connection = sqlite3.connect('work_data.db')
codes = ['2330','2317']
update_historical_data('2024-11-1','2024-11-10', codes, connection, api)

正在更新每日收盤行情...
20241031 找不到資料
20241101 每日收盤行情抓取完成，等待15秒
20241104 每日收盤行情抓取完成，等待15秒
20241105 每日收盤行情抓取完成，等待15秒
20241106 每日收盤行情抓取完成，等待15秒
20241107 每日收盤行情抓取完成，等待15秒
20241108 每日收盤行情抓取完成，等待15秒
正在更新逐筆交易...


NameError: name 'daily_targets' is not defined

In [20]:
ticks = {}
kbars = {}
volume_today = {}

In [21]:
def tbot(codes, is_Futures = False):
    import pandas as pd
    
    print('--- 啟動 Tbot #{} ---'.format(codes))

    tw_calendar = get_calendar('XTAI')
    
    date = pd.to_datetime(datetime.datetime.now().date())
    prev_trading_date = tw_calendar.previous_close(date).date()

    if date not in tw_calendar.opens:
        print('今日非交易日')
        return
    
    if is_Futures:
        code = ''.join(char for char in codes if char.isalpha())
    else:
        code = codes
    
    ticks[code] = pd.concat([get_ticks(connection, api, prev_trading_date, codes, is_Futures)[0],get_ticks(connection, api, date, codes, is_Futures)[0]])
    ticks[code] = ticks[code][['close','volume']]
    kbars[code] = ticks_to_kbars(ticks[code], interval='1Min')
    kbars[code] = kbars[code][date:]
    volume_today[code] = kbars[code]['volume'].sum()
    
    if is_Futures:
        api.quote.subscribe(api.Contracts.Futures.get(codes[0:3])[codes], 
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1 
                            )
        

    else:
        api.quote.subscribe(api.Contracts.Stocks[codes],
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1
                            )

    while True:

        time.sleep(1)

        current_time = datetime.datetime.now()

        if current_time.second == 0 :

            kbars[code] = ticks_to_kbars(ticks[code], interval='1Min')
            kbars[code] = kbars[code][prev_trading_date:]

            str_current_time = (current_time - datetime.timedelta(minutes=1)).strftime('%Y-%m-%d %H:%M:%S')

            try:
                current_kbars = kbars[code].loc[str_current_time]
            except:
                continue

            print(current_kbars)


In [25]:
tbot('2330')

--- 啟動 Tbot #2330 ---
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/2330 | Event: Subscribe or Unsubscribe ok
Exchange.TSE Tick(code='2330', datetime=datetime.datetime(2024, 11, 7, 13, 23, 31, 27045), open=Decimal('1050'), avg_price=Decimal('1063.86'), close=Decimal('1065'), high=Decimal('1075'), low=Decimal('1050'), amount=Decimal('1065000'), total_amount=Decimal('41900435000'), volume=1, total_volume=39385, tick_type=2, chg_type=2, price_chg=Decimal('5'), pct_chg=Decimal('0.47'), bid_side_total_vol=19631, ask_side_total_vol=19754, bid_side_total_cnt=4463, ask_side_total_cnt=3943, closing_oddlot_shares=0, fixed_trade_vol=0, suspend=0, simtrade=0, intraday_odd=0)
Exchange.TSE Tick(code='2330', datetime=datetime.datetime(2024, 11, 7, 13, 23, 31, 603012), open=Decimal('1050'), avg_price=Decimal('1063.86'), close=Decimal('1065'), high=Decimal('1075'), low=Decimal('1050'), amount=Decimal('1065000'), total_amount=Decimal('41901500000'), volume=1, total_volume=39386, tick_type

KeyboardInterrupt: 

Exchange.TSE Tick(code='2330', datetime=datetime.datetime(2024, 11, 7, 13, 24, 28, 80801), open=Decimal('1050'), avg_price=Decimal('1063.87'), close=Decimal('1065'), high=Decimal('1075'), low=Decimal('1050'), amount=Decimal('1065000'), total_amount=Decimal('42049800000'), volume=1, total_volume=39525, tick_type=2, chg_type=2, price_chg=Decimal('5'), pct_chg=Decimal('0.47'), bid_side_total_vol=19684, ask_side_total_vol=19841, bid_side_total_cnt=4486, ask_side_total_cnt=3992, closing_oddlot_shares=0, fixed_trade_vol=0, suspend=0, simtrade=0, intraday_odd=0)
Exchange.TSE Tick(code='2330', datetime=datetime.datetime(2024, 11, 7, 13, 24, 28, 88017), open=Decimal('1050'), avg_price=Decimal('1063.87'), close=Decimal('1070'), high=Decimal('1075'), low=Decimal('1050'), amount=Decimal('1070000'), total_amount=Decimal('42050870000'), volume=1, total_volume=39526, tick_type=1, chg_type=2, price_chg=Decimal('10'), pct_chg=Decimal('0.94'), bid_side_total_vol=19685, ask_side_total_vol=19841, bid_side

In [26]:
api.quote.unsubscribe(api.Contracts.Stocks['2330'],
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1
                            )

Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/2330 | Event: Subscribe or Unsubscribe ok
